# 재구조화 (1) - 피벗 테이블, Melt

데이터를 요약하여 재구조화 하는 형태

## #01. 준비과정

### [1] 패키지 참조

In [6]:
from pandas import read_excel, pivot_table, melt

### [2] 샘플 데이터 가져오기

서울, 인천, 부산에서 2005년, 2010년, 2015년에 조사한 가상의 인구수 데이터

단, 인천의 경우 2005년은 조사되지 않음

In [3]:
origin = read_excel("https://data.hossam.kr/pydata/city_people.xlsx")
origin

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,2632035,수도권


## #02. 피벗테이블 만들기

피벗테이블은 데이터프레임의 컬럼을 선정하여 인덱스와 변수로 나열하고 그에 따른 특정 데이터를 재배치하는 형태.
pandas 패키지의 `pivot_table()` 함수를 사용한다.

### [1] 기본 사용 방법

인덱스, 컬럼, 값으로 사용할 필드를 각각 지정하여 데이터를 재배치한다.

피벗 테이블 역시 데이터프레임이다.

> 도시와 연도에 따른 인구수 재배치


In [4]:
pivot1 = pivot_table(origin,            # 피벗할 데이터프레임
                     index = '도시',     # 행 위치에 들어갈 열
                     columns = '연도',   # 열 위치에 들어갈 열
                     values = '인구'     # 데이터로 사용할 열
)

print(type(pivot1))
pivot1

<class 'pandas.core.frame.DataFrame'>


연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,2632035.0,2890451.0


### [2] 중복 데이터의 집계 방법 지정하기

생성된 피벗 테이블을 이용하여 새로운 분석을 수행하지 않고 단순 출력만 할 경우 리턴받을 필요는 없다.

인덱스와 컬럼에 따른 value가 두 개 이상인 경우 집계방법을 지정해야 정확한 결과를 얻을 수 있다.

예제에서는 2010년과 2015년에 대한 수도권 데이터가 두 개씩 존재한다.

전체 인구수를 얻어야 하는 경우라면 합계를 구해야 한다.

In [ ]:
pivot_table(origin,             # 피벗할 데이터프레임
            index = '도시',     # 행 위치에 들어갈 열
            columns = '연도',   # 열 위치에 들어갈 열
            values = '인구',     # 데이터로 사용할 열
            aggfunc = 'sum'     # 데이터가 두 개 이상일 경우 집계함수 지정
)

### [3] 두 개 이상의 집계 함수 지정

집계 함수의 이름을 리스트로 설정한다.

In [ ]:
pivot_table(origin,
            index = '도시', 
            columns = '연도',
            values = '인구',
            aggfunc = ['mean', 'sum']
)

### [4] 복수 인덱스 지정

In [5]:
pivot_table(origin,
            index = ['지역', '연도'],
            columns = '도시',
            values = '인구',
            aggfunc =  ['mean', 'sum']
)

mean                              sum                      
도시               부산         서울         인천         부산         서울         인천
지역  연도                                                                    
경상권 2005  3512547.0        NaN        NaN  3512547.0        NaN        NaN
    2010  3393191.0        NaN        NaN  3393191.0        NaN        NaN
    2015  3448737.0        NaN        NaN  3448737.0        NaN        NaN
수도권 2005        NaN  9762546.0        NaN        NaN  9762546.0        NaN
    2010        NaN  9631482.0  2632035.0        NaN  9631482.0  2632035.0
    2015        NaN  9904312.0  2890451.0        NaN  9904312.0  2890451.0

## #03. melt

데이터 테이블의 컬럼 이름을 변수화 한 형태

피벗테이블을 분리한 것으로 볼 수 있다.


### [1] 샘플 피벗 테이블 생성

In [7]:
pivot_df = pivot_table(origin, 
                       index='연도', 
                       columns = '지역', 
                       values = '인구',
                       aggfunc = 'mean')
pivot_df

지역,경상권,수도권
연도,,
2005,3512547.0,9762546.0
2010,3393191.0,6131758.5
2015,3448737.0,6397381.5


### [2] 피벗 테이블 분리

#### (1) 데이터프레임의 인덱스를 일반 컬럼으로 설정

In [8]:
pivot_df2 = pivot_df.reset_index()
pivot_df2

지역,연도,경상권,수도권
0,2005,3512547.0,9762546.0
1,2010,3393191.0,6131758.5
2,2015,3448737.0,6397381.5


#### (2) 피벗 테이블의 분리

- id_vars: 인덱스로 사용할 컬럼이름. 반드시 컬럼만 가능 (인덱스 불가)
- value_vars: 분리할 컬럼 이름들

In [9]:
mdf = melt(pivot_df2, id_vars=['연도'], value_vars=['경상권', '수도권'])
mdf

,연도,지역,value
0,2005,경상권,3512547.0
1,2010,경상권,3393191.0
2,2015,경상권,3448737.0
3,2005,수도권,9762546.0
4,2010,수도권,6131758.5
5,2015,수도권,6397381.5


#### (3) 피벗 테이블 분리 및 필드 이름 지정

In [10]:
mdf = melt(pivot_df2, id_vars=['연도'], value_vars=['경상권', '수도권'], var_name='구분', value_name = '인구수')
mdf

,연도,구분,인구수
0,2005,경상권,3512547.0
1,2010,경상권,3393191.0
2,2015,경상권,3448737.0
3,2005,수도권,9762546.0
4,2010,수도권,6131758.5
5,2015,수도권,6397381.5
